<a href="https://colab.research.google.com/github/erikrozi/acmlab/blob/main/unsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in the Shared Google Drive
FOLDERNAME = 'Shared drives/Unsupervised'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/{}'.format(FOLDERNAME))

%cd /content/drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/Shared drives/Unsupervised


In [2]:
# Importing the standard ML libraries...
%load_ext autoreload
%autoreload 2

import pandas as pd                     # to process our data
import matplotlib.pyplot as plt         # graphing
import numpy as np                      # matrices

import torch
import torchvision                      # for MNIST dataset/working with images

import math

# take advantage of GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load Census Data

In [45]:
tax_returns_data = pd.read_csv("16zpallnoagi.csv")
cleaned_tax_data = pd.DataFrame(tax_returns_data, columns = ["ZIPCODE", "N1", "A02650"])
cleaned_tax_data = cleaned_tax_data[(cleaned_tax_data['ZIPCODE'] <= 97000) & (cleaned_tax_data['ZIPCODE'] >= 90000)] #remove zipcodes not in california
cleaned_tax_data

,ZIPCODE,N1,A02650
1640,90001,21670,619635
1641,90002,19890,563172
1642,90003,27290,730810
1643,90004,27810,1995586
1644,90005,15850,807743
...,...,...,...
5827,96853,300,7213
5828,96857,1000,24730
5829,96859,140,3424
5830,96860,1230,35506


In [46]:
tax_returns_data = pd.read_csv("16zpallnoagi.csv")
cleaned_tax_data_edit = pd.DataFrame(tax_returns_data, columns = ["ZIPCODE", "N1", "A02650"])
#cleaned_tax_data_edit = cleaned_tax_data_edit[(cleaned_tax_data_edit['ZIPCODE'] <= 97000) & (cleaned_tax_data_edit['ZIPCODE'] >= 90000)] #remove zipcodes not in california
cleaned_tax_data_edit.columns = ['zip', 'N1', 'A02650']
cleaned_tax_data_edit['aveincome'] = (cleaned_tax_data_edit['A02650'] / cleaned_tax_data_edit['N1']) * 1000
cleaned_tax_data_edit

,zip,N1,A02650,aveincome
1640,90001,21670,619635,28594.139363
1641,90002,19890,563172,28314.328808
1642,90003,27290,730810,26779.406376
1643,90004,27810,1995586,71757.856886
1644,90005,15850,807743,50961.703470
...,...,...,...,...
5827,96853,300,7213,24043.333333
5828,96857,1000,24730,24730.000000
5829,96859,140,3424,24457.142857
5830,96860,1230,35506,28866.666667


In [42]:
ziplation_data_edit = pd.read_csv("ziplatlon.csv", sep = ";")
parsed_ziplation_data_edit = ziplation_data_edit[(ziplation_data_edit['latitude'] <= 34.3) & (ziplation_data_edit['latitude'] >= 33.5) & (ziplation_data_edit['longitude'] >= -118.6) & (ziplation_data_edit['longitude'] <= -117.6)]
merged_zip_data = pd.merge(parsed_ziplation_data_edit, cleaned_tax_data_edit, how='left', on= 'zip')
merged_zip_data.dropna()

,zip,city,state,latitude,longitude,timezone,dst,geopoint,N1,A02650,aveincome
0,90038,Los Angeles,CA,34.089459,-118.32850,-8,1,"34.089459,-118.3285",13310.0,611675.0,45956.048084
1,90063,Los Angeles,CA,34.045161,-118.18650,-8,1,"34.045161,-118.1865",21330.0,720152.0,33762.400375
3,90301,Inglewood,CA,33.955913,-118.35868,-8,1,"33.955913,-118.35868",16690.0,599773.0,35936.069503
8,92840,Garden Grove,CA,33.785166,-117.93406,-8,1,"33.785166,-117.93406",25000.0,1113586.0,44543.440000
11,90220,Compton,CA,33.890566,-118.23666,-8,1,"33.890566,-118.23666",21890.0,789884.0,36084.239379
...,...,...,...,...,...,...,...,...,...,...,...
640,90504,Torrance,CA,33.870414,-118.33065,-8,1,"33.870414,-118.33065",16810.0,1112650.0,66189.767995
641,90640,Montebello,CA,34.014212,-118.11358,-8,1,"34.014212,-118.11358",29080.0,1365981.0,46973.211829
643,90620,Buena Park,CA,33.841014,-118.00969,-8,1,"33.841014,-118.00969",22330.0,1346431.0,60296.954769
644,91502,Burbank,CA,34.177606,-118.30954,-8,1,"34.177606,-118.30954",5530.0,308485.0,55783.905967


In [4]:
ziplation_data = pd.read_csv("ziplatlon.csv", sep = ";")
parsed_ziplation_data = ziplation_data[(ziplation_data['latitude'] <= 34.3) & (ziplation_data['latitude'] >= 33.5) & (ziplation_data['longitude'] >= -118.6) & (ziplation_data['longitude'] <= -117.6)]
parsed_ziplation_data

,zip,city,state,latitude,longitude,timezone,dst,geopoint
98,90038,Los Angeles,CA,34.089459,-118.328500,-8,1,"34.089459,-118.3285"
99,90063,Los Angeles,CA,34.045161,-118.186500,-8,1,"34.045161,-118.1865"
100,92709,Irvine,CA,33.640302,-117.769442,-8,1,"33.640302,-117.769442"
252,90301,Inglewood,CA,33.955913,-118.358680,-8,1,"33.955913,-118.35868"
256,92837,Fullerton,CA,33.640302,-117.769442,-8,1,"33.640302,-117.769442"
...,...,...,...,...,...,...,...,...
42980,90397,Inglewood,CA,33.786594,-118.298662,-8,1,"33.786594,-118.298662"
43080,91716,City Of Industry,CA,33.786594,-118.298662,-8,1,"33.786594,-118.298662"
43135,92702,Santa Ana,CA,33.640302,-117.769442,-8,1,"33.640302,-117.769442"
43136,92834,Fullerton,CA,33.640302,-117.769442,-8,1,"33.640302,-117.769442"


In [5]:
import util as util
import webmercator as webmercator

In [6]:
z = 14
pos = np.empty([1980, 5])
for x in range(2794, 2839):
  for y in range(6528, 6572):
    curPos = (x - 2794) * (6572 - 6528) + (y - 6528)
    pos[curPos, 0] = x
    pos[curPos, 1] = y
    pos[curPos, 2] = webmercator.lat(y, z)
    pos[curPos, 3] = webmercator.lon(x, z)
    pos[curPos, 4] = util.getElevation(pos[curPos, 2], pos[curPos, 3])
image_pos_data = pd.DataFrame({'x': pos[:,0].astype(int), 'y': pos[:,1].astype(int), 'z': 14, 'latitude': pos[:,2], 'longitude': pos[:,3], 'elevation': pos[:,4]})
image_pos_data['img'] = "14_" + image_pos_data['x'].astype(str) + "_" + image_pos_data['y'].astype(str) + ".jpg"
#A02650/N1

image_pos_data

,x,y,z,latitude,longitude,elevation,img
0,2794,6528,14,34.307144,-118.608398,660.0,14_2794_6528.jpg
1,2794,6529,14,34.288992,-118.608398,501.0,14_2794_6529.jpg
2,2794,6530,14,34.270836,-118.608398,392.0,14_2794_6530.jpg
3,2794,6531,14,34.252676,-118.608398,294.0,14_2794_6531.jpg
4,2794,6532,14,34.234512,-118.608398,280.0,14_2794_6532.jpg
...,...,...,...,...,...,...,...
1975,2838,6567,14,33.596319,-117.641602,231.0,14_2838_6567.jpg
1976,2838,6568,14,33.578015,-117.641602,151.0,14_2838_6568.jpg
1977,2838,6569,14,33.559707,-117.641602,178.0,14_2838_6569.jpg
1978,2838,6570,14,33.541395,-117.641602,136.0,14_2838_6570.jpg


eliminate ocean tiles

still need to: map zipcode to income, map latitude/longitude to zipcode

In [7]:
def calc_distance(lat1, long1, lat2, long2):
  xdist = long1 - long2
  ydist = lat1 - lat2
  return math.sqrt(xdist**2 + ydist**2)

In [11]:
np_ziplation = parsed_ziplation_data.to_numpy()
tile_zips = np.empty([image_pos_data.shape[0], 1])

np_ziplation.shape

#very inefficient lol
#for each image tile, loops through all the possible zipcodes, and chooses the one with the smallest distance
for i in range(image_pos_data.shape[0]):
  min_dist = 100000 #just made this an arbitrarily large number cuz iwas lazy
  for j in range(np_ziplation.shape[0]):
    zip_coords = np_ziplation[j][7].split(",") #splitting geopoint coords into latitude and longitude
    pt = image_pos_data['latitude'][i]
    cur_dist = calc_distance(pt, image_pos_data['longitude'][i], float(zip_coords[0]), float(zip_coords[1]))
    if cur_dist < min_dist:
      min_dist = cur_dist
      tile_zips[i] = np_ziplation[j][0]
image_pos_data['zipcode'] = tile_zips.astype(np.int32)
image_pos_data

KeyError: ignored

In [12]:
image_pos_data['zipcode'] = tile_zips.astype(np.int32)
image_pos_data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,x,y,z,latitude,longitude,elevation,img,zipcode
0,2794,6528,14,34.307144,-118.608398,660.0,14_2794_6528.jpg,91311
1,2794,6529,14,34.288992,-118.608398,501.0,14_2794_6529.jpg,91311
2,2794,6530,14,34.270836,-118.608398,392.0,14_2794_6530.jpg,91311
3,2794,6531,14,34.252676,-118.608398,294.0,14_2794_6531.jpg,91311
4,2794,6532,14,34.234512,-118.608398,280.0,14_2794_6532.jpg,91311
...,...,...,...,...,...,...,...,...
1975,2838,6567,14,33.596319,-117.641602,231.0,14_2838_6567.jpg,231
1976,2838,6568,14,33.578015,-117.641602,151.0,14_2838_6568.jpg,151
1977,2838,6569,14,33.559707,-117.641602,178.0,14_2838_6569.jpg,178
1978,2838,6570,14,33.541395,-117.641602,136.0,14_2838_6570.jpg,136


In [13]:
image_pos_data = image_pos_data[np.logical_not(image_pos_data['elevation'] == 0)]
image_pos_data

,x,y,z,latitude,longitude,elevation,img,zipcode
0,2794,6528,14,34.307144,-118.608398,660.0,14_2794_6528.jpg,91311
1,2794,6529,14,34.288992,-118.608398,501.0,14_2794_6529.jpg,91311
2,2794,6530,14,34.270836,-118.608398,392.0,14_2794_6530.jpg,91311
3,2794,6531,14,34.252676,-118.608398,294.0,14_2794_6531.jpg,91311
4,2794,6532,14,34.234512,-118.608398,280.0,14_2794_6532.jpg,91311
...,...,...,...,...,...,...,...,...
1975,2838,6567,14,33.596319,-117.641602,231.0,14_2838_6567.jpg,231
1976,2838,6568,14,33.578015,-117.641602,151.0,14_2838_6568.jpg,151
1977,2838,6569,14,33.559707,-117.641602,178.0,14_2838_6569.jpg,178
1978,2838,6570,14,33.541395,-117.641602,136.0,14_2838_6570.jpg,136


In [ ]:
#would be better if i mapped income directly to zipcode
avg_income = np.empty([image_pos_data.shape[0]])
for i in range(image_pos_data.shape[0]):
  income_sum = cleaned_tax_data.loc[cleaned_tax_data.ZIPCODE == image_pos_data['zipcode'][i], 'A02650']
  income_returns = cleaned_tax_data.loc[cleaned_tax_data.ZIPCODE == image_pos_data['zipcode'][i], 'N1']
  avg_income[i] = income_sum / income_returns

avg_income
  

IndexError: ignored

In [ ]:
image_pos_data.shape

(1457, 8)

# Load the imagery data


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class TileImageryDataset(Dataset):
  """Tile imagery dataset."""

  def __init__(self, images_zip_path):
    # Load into tensors

    # Perform data augmentation

# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
  def load_images():
    tensors = []
    for x in range(2794, 2839):
      for y in range(6528, 6572):
        image = Image.open(f"images/14_{x}_{y}.jpg").convert("RGB")
        # TODO change
        #jaden
        data = asarray(image) #convert to np array, could also do np.array(Image.open...)
        data = torch.from_numpy(data) #convert to tensor
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        input_tensor = preprocess(image)
        # tensory push thing?

IndentationError: ignored